In [1]:
import os

In [2]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction/research'

In [3]:
os.chdir("../")

In [34]:
%pwd

'/home/tejas/MLProj/Thyroid-Disease-Prediction'

In [35]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    categorical_algorithm: str
    l1_regularization: float
    l2_categorical_regularization: float
    l2_regularization: float
    max_depth: int
    num_trees: int
    target_column: str

In [36]:

from ThyroidProject.constants import *
from ThyroidProject.utils.common import read_yaml, create_directories

In [37]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH,
                 schema_filepath=SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_trainer_config(self) -> ModelTrainerConfig:
        """
        This function returns the ModelTrainerConfig object that contains the configuration for the model training process.

        Args:
            None

        Returns:
            ModelTrainerConfig: The ModelTrainerConfig object containing the configuration for the model training process

        """
        config = self.config.model_trainer
        params = self.params.GradientBoostedTreesLearner
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_name=config.model_name,
            categorical_algorithm=params.categorical_algorithm,
            l1_regularization=params.l1_regularization,
            l2_categorical_regularization=params.l2_categorical_regularization,
            l2_regularization=params.l2_regularization,
            max_depth=params.max_depth,
            num_trees=params.num_trees,
            target_column=schema.name,
        )
        return model_trainer_config

In [38]:
import pandas as pd
import os
from ThyroidProject import logger
import ydf

In [39]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    def train(self):
        """
        trains the model using the training data and saves it to the artifacts directory
        """
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        learner = ydf.GradientBoostedTreesLearner(
            categorical_algorithm=self.config.categorical_algorithm,
            l1_regularization=self.config.l1_regularization,
            l2_categorical_regularization=self.config.l2_categorical_regularization,
            l2_regularization=self.config.l2_regularization,
            max_depth=self.config.max_depth,
            num_trees=self.config.num_trees,
            label=self.config.target_column
        )
        model = learner.train(train_data)
        assert model.task() == ydf.Task.CLASSIFICATION
        results = model.evaluate(test_data)
        logger.info(f"The results of the model evaluation are: {results.accuracy:.4f}")
        model.save(os.path.join(self.config.root_dir, self.config.model_name))

In [40]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2024-01-27 02:45:48,381:INFO:common:yaml file: config/config.yaml loaded successfully]
[2024-01-27 02:45:48,382:INFO:common:yaml file: params.yaml loaded successfully]
[2024-01-27 02:45:48,383:INFO:common:yaml file: schema.yaml loaded successfully]
[2024-01-27 02:45:48,384:INFO:common:created directory at :artifacts]
[2024-01-27 02:45:48,384:INFO:common:created directory at :artifacts/model_trainer]
[2024-01-27 02:45:48,396:INFO:generic_learner:Use 16 thread(s) for training]
Train model on 5756 examples
Model trained in 0:00:00.399516
[2024-01-27 02:45:48,817:INFO:2531560583:The results of the model evaluation are: 0.9870]
